In [1]:
%matplotlib inline

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import random

import sys
sys.path.append('./python')
caffe_root = '../../'  # this file is expected to be in {caffe_root}/examples/coco_caption
sys.path.insert(0, caffe_root + 'python')
import caffe

caffe.set_mode_gpu()

sys.path.append('./examples/char-rnn')

In [3]:
!head ../../examples/char-rnn/vocabulary.txt

F
i
r
s
t
 
C
z
e
n


In [4]:
vocabulary = ['<EOS>'] + [line.strip() for line in
                          open('../../examples/char-rnn/vocab_reddit.txt').readlines()]
print len(vocabulary)

657


In [5]:
iter_num = 21550
net = caffe.Net('../../examples/char-rnn/reddit_deploy.prototxt',
                '../../examples/char-rnn/reddit-rnn_iter_%d.caffemodel' % iter_num, caffe.TEST)
print net.blobs['probs'].data.shape

(1, 1, 657)


In [6]:
def softmax(softmax_inputs, temp):
    shifted_inputs = softmax_inputs - softmax_inputs.max()
    exp_outputs = np.exp(temp * shifted_inputs)
    exp_outputs_sum = exp_outputs.sum()
    if np.isnan(exp_outputs_sum):
        return exp_outputs * float('nan')
    assert exp_outputs_sum > 0
    if np.isinf(exp_outputs_sum):
        return np.zeros_like(exp_outputs)
    eps_sum = 1e-20
    return exp_outputs / max(exp_outputs_sum, eps_sum)

def random_choice_from_probs(softmax_inputs, temp=1):
    # temperature of infinity == take the max
    if temp == float('inf'):
        return np.argmax(softmax_inputs)
    probs = softmax(softmax_inputs, temp)
    r = random.random()
    cum_sum = 0.
    for i, p in enumerate(probs):
        cum_sum += p
        if cum_sum >= r: return i
    return 1  # return UNK?

In [7]:
def generate_sentence(net, input_char, temp=float('inf'), output='predict', max_words=200):
    cont_input = np.array([0])
    word_input = np.array([vocabulary.index(input_char)])
    sentence = []
    while len(sentence) < max_words and (not sentence or sentence[-1] != 0):
        net.forward(cont=cont_input, input=word_input)
        output_preds = net.blobs[output].data[0, 0, :]
        sentence.append(random_choice_from_probs(output_preds, temp=temp))
        cont_input[0] = 1
        word_input[0] = sentence[-1]
    return sentence

def charstostr(char_arr):
    out = ''
    for i in [vocabulary[index] for index in char_arr]:
        if i == '':
            i = ' '
        if i == '\\n':
            i = '\n'
        out += i
        
    return out

In [8]:
choice = random.randint(0, len(vocabulary) - 1)
start_char = vocabulary[choice]
print 'Chosen %s as startup char with idx %d' % (start_char, choice)

Chosen \u8b66 as startup char with idx 432


In [9]:
sentence = generate_sentence(net, start_char)
print sentence
print charstostr(sentence)

[3, 18, 21, 10, 3, 36, 11, 19, 34, 6, 10, 31, 3, 14, 5, 3, 18, 21, 7, 18, 3, 18, 21, 10, 3, 36, 11, 19, 34, 6, 10, 31, 3, 14, 5, 3, 18, 21, 7, 18, 3, 18, 21, 10, 3, 36, 11, 19, 34, 6, 10, 31, 3, 14, 5, 3, 18, 21, 7, 18, 3, 18, 21, 10, 3, 36, 11, 19, 34, 6, 10, 31, 3, 14, 5, 3, 18, 21, 7, 18, 3, 18, 21, 10, 3, 36, 11, 19, 34, 6, 10, 31, 3, 14, 5, 3, 18, 21, 7, 18, 3, 18, 21, 10, 3, 36, 11, 19, 34, 6, 10, 31, 3, 14, 5, 3, 18, 21, 7, 18, 3, 18, 21, 10, 3, 36, 11, 19, 34, 6, 10, 31, 3, 14, 5, 3, 18, 21, 7, 18, 3, 18, 21, 10, 3, 36, 11, 19, 34, 6, 10, 31, 3, 14, 5, 3, 18, 21, 7, 18, 3, 18, 21, 10, 3, 36, 11, 19, 34, 6, 10, 31, 3, 14, 5, 3, 18, 21, 7, 18, 3, 18, 21, 10, 3, 36, 11, 19, 34, 6, 10, 31, 3, 14, 5, 3, 18, 21, 7, 18]
 the problem is that the problem is that the problem is that the problem is that the problem is that the problem is that the problem is that the problem is that the problem is that the problem is that


In [10]:
sentence = generate_sentence(net, start_char)
print sentence
print charstostr(sentence)

[3, 18, 21, 10, 3, 36, 11, 19, 34, 6, 10, 31, 3, 14, 5, 3, 18, 21, 7, 18, 3, 18, 21, 10, 3, 36, 11, 19, 34, 6, 10, 31, 3, 14, 5, 3, 18, 21, 7, 18, 3, 18, 21, 10, 3, 36, 11, 19, 34, 6, 10, 31, 3, 14, 5, 3, 18, 21, 7, 18, 3, 18, 21, 10, 3, 36, 11, 19, 34, 6, 10, 31, 3, 14, 5, 3, 18, 21, 7, 18, 3, 18, 21, 10, 3, 36, 11, 19, 34, 6, 10, 31, 3, 14, 5, 3, 18, 21, 7, 18, 3, 18, 21, 10, 3, 36, 11, 19, 34, 6, 10, 31, 3, 14, 5, 3, 18, 21, 7, 18, 3, 18, 21, 10, 3, 36, 11, 19, 34, 6, 10, 31, 3, 14, 5, 3, 18, 21, 7, 18, 3, 18, 21, 10, 3, 36, 11, 19, 34, 6, 10, 31, 3, 14, 5, 3, 18, 21, 7, 18, 3, 18, 21, 10, 3, 36, 11, 19, 34, 6, 10, 31, 3, 14, 5, 3, 18, 21, 7, 18, 3, 18, 21, 10, 3, 36, 11, 19, 34, 6, 10, 31, 3, 14, 5, 3, 18, 21, 7, 18]
 the problem is that the problem is that the problem is that the problem is that the problem is that the problem is that the problem is that the problem is that the problem is that the problem is that


In [11]:
sentence = generate_sentence(net, start_char, temp=1.0)
print sentence
print charstostr(sentence)

[11, 10, 8, 44, 21, 12, 3, 14, 18, 3, 7, 6, 5, 19, 3, 9, 11, 10, 5, 5, 10, 9, 3, 7, 17, 18, 19, 31, 7, 18, 14, 44, 7, 6, 6, 23, 12, 3, 7, 11, 10, 3, 23, 19, 17, 3, 15, 21, 10, 11, 10, 3, 18, 21, 19, 5, 10, 3, 15, 19, 17, 6, 9, 3, 6, 19, 19, 33, 3, 7, 18, 3, 23, 19, 17, 11, 3, 19, 15, 8, 3, 44, 7, 17, 5, 14, 8, 20, 3, 19, 11, 3, 5, 21, 19, 11, 18, 3, 11, 10, 44, 10, 14, 30, 10, 11, 28, 3, 40, 17, 18, 3, 18, 21, 14, 5, 3, 14, 5, 8, 29, 18, 3, 18, 21, 7, 18, 3, 31, 23, 3, 8, 7, 18, 17, 11, 7, 6, 3, 14, 5, 5, 17, 10, 12, 3, 14, 5, 3, 9, 10, 32, 14, 8, 14, 18, 10, 6, 23, 3, 5, 6, 7, 30, 10, 5, 12, 3, 20, 19, 3, 18, 19, 3, 18, 21, 10, 3, 20, 23, 31, 3, 14, 5, 3, 7, 34, 19, 17, 18, 3, 86, 3, 36, 11, 19, 18, 10, 5, 18]
rench, it also dressed automatically, are you where those would look at your own causing or short receiver. But this isn't that my natural issue, is definitely slaves, go to the gym is about 7 protest


In [16]:
sentence = generate_sentence(net, start_char, temp=1.0)
print sentence
print charstostr(sentence)

[20, 14, 43, 14, 43, 12, 3, 13, 17, 11, 11, 23, 3, 45, 11, 14, 34, 10, 3, 56, 60, 28, 22, 22, 50, 7, 31, 36, 51, 8, 34, 5, 36, 51, 22, 22, 45, 21, 14, 5, 3, 14, 5, 3, 7, 3, 5, 18, 7, 8, 9, 7, 11, 9, 3, 32, 19, 19, 33, 3, 14, 32, 3, 18, 21, 10, 23, 3, 9, 14, 9, 8, 29, 18, 3, 11, 10, 7, 6, 6, 23, 3, 10, 8, 6, 10, 44, 18, 3, 31, 10, 3, 34, 10, 3, 18, 21, 10, 3, 32, 14, 11, 5, 18, 3, 7, 8, 20, 10, 11, 3, 19, 8, 10, 5, 28, 3, 4, 18, 3, 15, 19, 11, 33, 5, 28, 3, 45, 21, 14, 5, 3, 15, 7, 5, 3, 7, 3, 17, 8, 14, 53, 17, 10, 12, 3, 18, 21, 10, 23, 3, 15, 10, 11, 10, 3, 24, 75, 3, 23, 10, 7, 11, 5, 3, 19, 6, 9, 28, 1, 22, 1, 61, 19, 11, 3, 31, 10, 3, 15, 11, 14, 18, 14, 20, 10, 3, 10, 77, 18, 11, 7, 44, 18, 19, 11, 39, 3, 45, 21]
giziz, Murry Tribe OP.

&amp;nbsp;

This is a standard fook if they didn't really enlect me be the first anger ones. It works. This was a unique, they were 12 years old."
"For me writige extractor! Th


In [17]:
sentence = generate_sentence(net, start_char, temp=1.5)
print sentence
print charstostr(sentence)

[3, 44, 19, 8, 18, 11, 19, 6, 3, 19, 32, 3, 18, 21, 10, 3, 31, 7, 14, 8, 3, 18, 10, 11, 31, 5, 3, 19, 32, 3, 5, 18, 7, 18, 10, 52, 15, 14, 6, 6, 3, 34, 10, 3, 11, 10, 7, 6, 6, 23, 3, 31, 10, 7, 8, 18, 3, 18, 19, 3, 34, 17, 23, 3, 7, 8, 23, 18, 21, 14, 8, 20, 3, 10, 6, 5, 10, 3, 36, 11, 19, 20, 11, 7, 31, 31, 14, 8, 20, 3, 7, 8, 9, 3, 5, 10, 10, 3, 21, 19, 15, 3, 31, 7, 8, 23, 3, 36, 10, 19, 36, 6, 10, 3, 5, 21, 19, 17, 6, 9, 3, 9, 19, 3, 7, 8, 23, 18, 21, 14, 8, 20, 3, 10, 30, 10, 11, 23, 19, 8, 10, 3, 10, 6, 5, 10, 3, 18, 19, 3, 34, 10, 3, 5, 19, 3, 31, 17, 44, 21, 3, 34, 10, 18, 18, 10, 11, 3, 18, 21, 7, 8, 3, 19, 8, 10, 3, 18, 21, 14, 8, 20, 3, 18, 19, 3, 9, 19, 3, 18, 21, 7, 18, 3, 14, 32, 3, 4, 3, 44]
 control of the main terms of state-will be really meant to buy anything else programming and see how many people should do anything everyone else to be so much better than one thing to do that if I c


In [18]:
sentence = generate_sentence(net, start_char, temp=1.5)
print sentence
print charstostr(sentence)

[8, 10, 9, 28, 22, 22, 82, 3, 63, 83, 65, 27, 3, 3, 32, 19, 11, 3, 18, 21, 10, 3, 11, 7, 18, 10, 9, 3, 81, 10, 15, 3, 55, 19, 30, 10, 3, 2, 19, 19, 20, 6, 10, 3, 2, 7, 31, 10, 36, 11, 7, 20, 20, 10, 11, 10, 11, 5, 3, 7, 8, 9, 3, 60, 57, 60, 3, 14, 5, 3, 5, 18, 14, 6, 6, 3, 19, 8, 6, 23, 3, 7, 3, 44, 6, 19, 5, 10, 11, 28, 3, 3, 40, 17, 18, 3, 15, 10, 3, 8, 10, 10, 9, 3, 18, 19, 3, 5, 18, 7, 23, 3, 14, 8, 3, 7, 3, 11, 10, 7, 6, 3, 11, 10, 7, 9, 14, 8, 20, 3, 18, 21, 7, 18, 3, 18, 21, 10, 11, 10, 3, 7, 11, 10, 3, 5, 7, 23, 14, 8, 20, 3, 18, 21, 10, 23, 3, 44, 19, 17, 8, 18, 3, 7, 3, 5, 17, 36, 10, 11, 3, 44, 19, 31, 36, 6, 10, 77, 3, 9, 10, 18, 10, 11, 31, 14, 8, 14, 8, 20, 3, 7, 18, 3, 18, 21, 10, 3, 5, 7]
ned.

* ~690  for the rated New Love Google Gamepraggerers and PVP is still only a closer.  But we need to stay in a real reading that there are saying they count a super complex determining at the sa


In [19]:
sentence = generate_sentence(net, start_char, temp=3.0)
print sentence
print charstostr(sentence)

[39, 22, 1, 46, 19, 15, 3, 9, 19, 3, 23, 19, 17, 3, 18, 21, 14, 8, 33, 3, 18, 21, 10, 23, 3, 15, 19, 17, 6, 9, 3, 21, 7, 30, 10, 3, 34, 10, 10, 8, 3, 7, 3, 6, 19, 8, 20, 3, 18, 14, 31, 10, 3, 18, 19, 3, 5, 10, 10, 3, 14, 32, 3, 23, 19, 17, 3, 15, 7, 8, 18, 3, 18, 19, 3, 5, 18, 7, 11, 18, 3, 18, 21, 10, 3, 44, 19, 8, 18, 11, 19, 6, 3, 19, 32, 3, 18, 21, 10, 3, 11, 10, 7, 5, 19, 8, 5, 3, 18, 21, 7, 18, 3, 7, 11, 10, 3, 19, 8, 3, 18, 21, 10, 3, 5, 7, 31, 10, 3, 18, 14, 31, 10, 3, 18, 19, 3, 7, 5, 33, 3, 18, 21, 10, 3, 18, 21, 14, 8, 20, 5, 3, 18, 21, 7, 18, 3, 7, 11, 10, 3, 36, 11, 10, 18, 18, 23, 3, 31, 17, 44, 21, 3, 7, 5, 3, 4, 3, 11, 10, 7, 6, 6, 23, 3, 34, 10, 3, 7, 3, 36, 11, 19, 34, 6, 10, 31, 3, 15, 14]
!
"How do you think they would have been a long time to see if you want to start the control of the reasons that are on the same time to ask the things that are pretty much as I really be a problem wi


In [20]:
sentence = generate_sentence(net, start_char, temp=3.0)
print sentence
print charstostr(sentence)

[3, 5, 18, 7, 18, 10, 5, 3, 18, 19, 3, 7, 30, 19, 14, 9, 3, 18, 21, 10, 3, 44, 21, 7, 8, 44, 10, 3, 18, 19, 3, 5, 10, 10, 3, 14, 32, 3, 18, 21, 10, 23, 3, 15, 7, 8, 18, 3, 18, 19, 3, 34, 10, 3, 7, 3, 31, 7, 8, 3, 15, 21, 19, 3, 11, 10, 7, 6, 14, 43, 10, 9, 3, 18, 21, 7, 18, 3, 14, 18, 3, 14, 5, 3, 16, 17, 5, 18, 3, 7, 3, 31, 7, 5, 5, 14, 30, 10, 3, 20, 7, 31, 10, 3, 18, 21, 7, 18, 3, 15, 10, 3, 9, 19, 8, 29, 18, 3, 21, 7, 30, 10, 3, 7, 3, 5, 10, 44, 19, 8, 9, 3, 36, 6, 7, 23, 10, 11, 3, 15, 21, 10, 8, 3, 18, 21, 10, 23, 3, 44, 19, 17, 6, 9, 3, 21, 7, 30, 10, 3, 5, 10, 10, 8, 3, 7, 3, 32, 10, 15, 3, 19, 11, 3, 5, 19, 31, 10, 18, 21, 14, 8, 20, 3, 18, 19, 3, 34, 10, 3, 7, 3, 30, 10, 11, 23, 3, 5, 18, 11]
 states to avoid the chance to see if they want to be a man who realized that it is just a massive game that we don't have a second player when they could have seen a few or something to be a very str


In [21]:
sentence = generate_sentence(net, start_char, temp=10.0)
print sentence
print charstostr(sentence)

[3, 18, 21, 10, 3, 36, 19, 14, 8, 18, 3, 14, 5, 3, 18, 21, 7, 18, 3, 18, 21, 10, 3, 36, 11, 19, 34, 6, 10, 31, 3, 14, 5, 3, 18, 21, 7, 18, 3, 18, 21, 10, 3, 36, 11, 19, 34, 6, 10, 31, 3, 14, 5, 3, 18, 21, 7, 18, 3, 18, 21, 10, 3, 36, 11, 19, 34, 6, 10, 31, 3, 14, 5, 3, 18, 21, 7, 18, 3, 18, 21, 10, 3, 36, 11, 19, 34, 6, 10, 31, 3, 14, 5, 3, 18, 21, 7, 18, 3, 18, 21, 10, 3, 36, 11, 19, 34, 6, 10, 31, 3, 14, 5, 3, 18, 21, 7, 18, 3, 18, 21, 10, 3, 18, 11, 7, 9, 10, 3, 9, 10, 44, 14, 5, 14, 19, 8, 3, 14, 5, 3, 18, 21, 10, 3, 19, 8, 6, 23, 3, 18, 21, 14, 8, 20, 3, 18, 21, 7, 18, 3, 15, 7, 5, 3, 7, 3, 34, 14, 18, 3, 19, 32, 3, 7, 3, 44, 19, 8, 18, 11, 7, 44, 18, 3, 32, 19, 11, 3, 18, 21, 10, 3, 11, 10, 5, 18, 3, 19, 32]
 the point is that the problem is that the problem is that the problem is that the problem is that the problem is that the trade decision is the only thing that was a bit of a contract for the rest of


In [22]:
sentence = generate_sentence(net, start_char, temp=1.0)
print sentence
print charstostr(sentence)

[22, 1, 81, 19, 18, 3, 5, 17, 11, 10, 3, 17, 18, 10, 5, 12, 3, 36, 10, 11, 5, 19, 8, 7, 6, 3, 32, 7, 44, 18, 14, 19, 8, 3, 11, 10, 6, 7, 18, 10, 9, 3, 18, 19, 3, 10, 30, 14, 9, 10, 8, 44, 10, 3, 32, 19, 11, 3, 44, 21, 10, 7, 36, 10, 11, 28, 1, 22, 1, 2, 19, 19, 9, 3, 8, 19, 18, 14, 44, 10, 3, 7, 5, 3, 7, 8, 23, 3, 14, 9, 10, 7, 28, 3, 1, 22, 1, 45, 21, 7, 18, 29, 5, 3, 24, 27, 59, 76, 27, 28, 3, 47, 14, 18, 21, 3, 14, 9, 52, 6, 7, 8, 9, 3, 7, 18, 3, 21, 14, 5, 3, 11, 19, 34, 19, 18, 3, 18, 19, 3, 76, 22, 6, 23, 6, 52, 25, 3, 5, 17, 36, 36, 23, 3, 34, 6, 17, 10, 3, 14, 5, 18, 3, 6, 14, 33, 10, 3, 54, 30, 26, 3, 45, 19, 17, 44, 21, 36, 7, 9, 1, 22, 90, 22, 21, 18, 18, 36, 41, 59, 59, 5, 18, 7, 18, 14, 44, 59, 20, 17]

"Not sure utes, personal faction related to evidence for cheaper."
"Good notice as any idea. "
"That's 10/30. With id-land at his robot to 3
lyl-8 suppy blue ist like Ev4 Touchpad"
\r
http://static/gu


In [24]:
sentence = generate_sentence(net, start_char, temp=0.8)
print sentence
print charstostr(sentence)

[11, 7, 9, 10, 3, 9, 10, 31, 7, 8, 9, 3, 8, 19, 3, 36, 7, 8, 10, 18, 3, 6, 14, 32, 10, 3, 14, 5, 3, 7, 3, 11, 19, 23, 7, 6, 3, 19, 8, 3, 35, 21, 14, 8, 14, 5, 21, 14, 8, 10, 5, 3, 13, 25, 28, 3, 4, 29, 31, 3, 10, 9, 14, 44, 6, 10, 9, 3, 7, 8, 9, 3, 9, 14, 10, 3, 5, 10, 7, 18, 5, 3, 19, 11, 3, 14, 5, 3, 18, 21, 7, 18, 3, 18, 21, 10, 11, 10, 29, 5, 3, 8, 19, 18, 21, 14, 8, 20, 3, 18, 21, 19, 17, 20, 21, 3, 19, 21, 12, 3, 18, 21, 10, 23, 29, 6, 6, 3, 11, 17, 14, 8, 3, 19, 17, 11, 3, 9, 11, 36, 3, 14, 8, 18, 11, 19, 22, 18, 21, 7, 18, 3, 15, 19, 17, 6, 9, 8, 18, 3, 9, 19, 31, 10, 3, 18, 21, 10, 3, 75, 3, 5, 7, 31, 10, 3, 5, 44, 19, 15, 10, 9, 3, 19, 11, 3, 21, 10, 11, 10, 28, 3, 1, 22, 75, 27, 24, 28, 3, 4]
rade demand no panet life is a royal on Chinishines M8. I'm edicled and die seats or is that there's nothing though oh, they'll ruin our drp intro
that wouldnt dome the 2 same scowed or here. "
201. I


In [ ]:
sentence = generate_sentence(net, start_char, temp=0.8)
print sentence
print charstostr(sentence)

In [ ]:
sentence = generate_sentence(net, start_char, temp=0.6)
print sentence
print charstostr(sentence)

In [ ]:
sentence = generate_sentence(net, start_char, temp=0.5)
print sentence
print charstostr(sentence)